# OpenAI APIs - Completions

SGLang provides OpenAI-compatible APIs to enable a smooth transition from OpenAI services to self-hosted local models.
A complete reference for the API is available in the [OpenAI API Reference](https://platform.openai.com/docs/api-reference).

This tutorial covers the following popular APIs:

- `chat/completions`
- `completions`
- `batches`

Check out other tutorials to learn about vision APIs for vision-language models and embedding APIs for embedding models.

## Launch A Server

This code block is equivalent to executing 

```bash
python -m sglang.launch_server --model-path meta-llama/Meta-Llama-3.1-8B-Instruct \
--port 30000 --host 0.0.0.0
```

in your terminal and wait for the server to be ready.

In [1]:
from sglang.utils import (
    execute_shell_command,
    wait_for_server,
    terminate_process,
    print_highlight,
)

server_process = execute_shell_command(
    "python -m sglang.launch_server --model-path meta-llama/Meta-Llama-3.1-8B-Instruct --port 30000 --host 0.0.0.0"
)

wait_for_server("http://localhost:30000")

[2024-11-04 09:43:43] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3.1-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, context_length=None, device='cuda', served_model_name='meta-llama/Meta-Llama-3.1-8B-Instruct', chat_template=None, is_embedding=False, host='0.0.0.0', port=30000, mem_fraction_static=0.88, max_running_requests=None, max_total_tokens=None, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='lpm', schedule_conservativeness=1.0, tp_size=1, stream_interval=1, random_seed=615367455, constrained_json_whitespace_pattern=None, decode_log_interval=40, log_level='info', log_level_http=None, log_requests=False, show_time_cost=False, api_key=None, file_storage_pth='SGLang_storage', enable_cache_report=False, watchdog_timeout=600, dp_size=1, load_balance_method='round_robin', dist_i

[2024-11-04 09:43:58 TP0] Init torch distributed begin.


[2024-11-04 09:43:59 TP0] Load weight begin. avail mem=78.59 GB


[2024-11-04 09:43:59 TP0] lm_eval is not installed, GPTQ may not be usable


INFO 11-04 09:44:00 weight_utils.py:243] Using model weights format ['*.safetensors']


...

Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.53it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.38it/s]

[2024-11-04 09:44:03 TP0] Load weight end. type=LlamaForCausalLM, dtype=torch.bfloat16, avail mem=63.50 GB
[2024-11-04 09:44:03 TP0] Memory pool end. avail mem=8.37 GB
[2024-11-04 09:44:03 TP0] Capture cuda graph begin. This can take up to several minutes.


[2024-11-04 09:44:11 TP0] max_total_num_tokens=442913, max_prefill_tokens=16384, max_running_requests=2049, context_len=131072


[2024-11-04 09:44:12] INFO:     Started server process [3186057]
[2024-11-04 09:44:12] INFO:     Waiting for application startup.
[2024-11-04 09:44:12] INFO:     Application startup complete.
[2024-11-04 09:44:12] INFO:     Uvicorn running on http://0.0.0.0:30000 (Press CTRL+C to quit)


[2024-11-04 09:44:13] INFO:     127.0.0.1:43108 - "GET /v1/models HTTP/1.1" 200 OK
[2024-11-04 09:44:13] INFO:     127.0.0.1:43124 - "GET /get_model_info HTTP/1.1" 200 OK
[2024-11-04 09:44:13 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, cache hit rate: 0.00%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2024-11-04 09:44:13] INFO:     127.0.0.1:43138 - "POST /generate HTTP/1.1" 200 OK
[2024-11-04 09:44:13] The server is fired up and ready to roll!


## Chat Completions

### Usage

The server fully implements the OpenAI API.
It will automatically apply the chat template specified in the Hugging Face tokenizer, if one is available.
You can also specify a custom chat template with `--chat-template` when launching the server.

In [2]:
import openai

client = openai.Client(base_url="http://127.0.0.1:30000/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2024-11-04 09:44:18 TP0] Prefill batch. #new-seq: 1, #new-token: 42, #cached-token: 1, cache hit rate: 2.00%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2024-11-04 09:44:18 TP0] Decode batch. #running-req: 1, #token: 76, token usage: 0.00, gen throughput (token/s): 5.73, #queue-req: 0
[2024-11-04 09:44:18] INFO:     127.0.0.1:48578 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### Parameters

The chat completions API accepts OpenAI Chat Completions API's parameters. Refer to [OpenAI Chat Completions API](https://platform.openai.com/docs/api-reference/chat/create) for more details.

Here is an example of a detailed chat completion request:

In [3]:
response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {
            "role": "system",
            "content": "You are a knowledgeable historian who provides concise responses.",
        },
        {"role": "user", "content": "Tell me about ancient Rome"},
        {
            "role": "assistant",
            "content": "Ancient Rome was a civilization centered in Italy.",
        },
        {"role": "user", "content": "What were their major achievements?"},
    ],
    temperature=0.3,  # Lower temperature for more focused responses
    max_tokens=128,  # Reasonable length for a concise response
    top_p=0.95,  # Slightly higher for better fluency
    presence_penalty=0.2,  # Mild penalty to avoid repetition
    frequency_penalty=0.2,  # Mild penalty for more natural language
    n=1,  # Single response is usually more stable
    seed=42,  # Keep for reproducibility
)

print_highlight(response.choices[0].message.content)

[2024-11-04 09:44:18 TP0] Prefill batch. #new-seq: 1, #new-token: 51, #cached-token: 25, cache hit rate: 20.63%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2024-11-04 09:44:19 TP0] Decode batch. #running-req: 1, #token: 107, token usage: 0.00, gen throughput (token/s): 109.19, #queue-req: 0


[2024-11-04 09:44:19 TP0] Decode batch. #running-req: 1, #token: 147, token usage: 0.00, gen throughput (token/s): 132.10, #queue-req: 0


[2024-11-04 09:44:19 TP0] Decode batch. #running-req: 1, #token: 187, token usage: 0.00, gen throughput (token/s): 131.19, #queue-req: 0
[2024-11-04 09:44:19] INFO:     127.0.0.1:48578 - "POST /v1/chat/completions HTTP/1.1" 200 OK


Streaming mode is also supported.

In [4]:
stream = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[{"role": "user", "content": "Say this is a test"}],
    stream=True,
)
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

[2024-11-04 09:44:19] INFO:     127.0.0.1:48578 - "POST /v1/chat/completions HTTP/1.1" 200 OK
[2024-11-04 09:44:19 TP0] Prefill batch. #new-seq: 1, #new-token: 10, #cached-token: 30, cache hit rate: 33.73%, token usage: 0.00, #running-req: 0, #queue-req: 0
It looks like you're ready to begin

 a test. Is there a specific subject or topic you'd like to focus on[2024-11-04 09:44:20 TP0] Decode batch. #running-req: 1, #token: 64, token usage: 0.00, gen throughput (token/s): 124.49, #queue-req: 0
, or would

 you like me to generate a random set of questions

?

## Completions

### Usage
Completions API is similar to Chat Completions API, but without the `messages` parameter or chat templates.

In [5]:
response = client.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    prompt="List 3 countries and their capitals.",
    temperature=0,
    max_tokens=64,
    n=1,
    stop=None,
)

print_highlight(f"Response: {response}")

[2024-11-04 09:44:20 TP0] Prefill batch. #new-seq: 1, #new-token: 8, #cached-token: 1, cache hit rate: 32.57%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2024-11-04 09:44:20 TP0] Decode batch. #running-req: 1, #token: 35, token usage: 0.00, gen throughput (token/s): 132.62, #queue-req: 0


[2024-11-04 09:44:20] INFO:     127.0.0.1:48578 - "POST /v1/completions HTTP/1.1" 200 OK


### Parameters

The completions API accepts OpenAI Completions API's parameters.  Refer to [OpenAI Completions API](https://platform.openai.com/docs/api-reference/completions/create) for more details.

Here is an example of a detailed completions request:

In [6]:
response = client.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    prompt="Write a short story about a space explorer.",
    temperature=0.7,  # Moderate temperature for creative writing
    max_tokens=150,  # Longer response for a story
    top_p=0.9,  # Balanced diversity in word choice
    stop=["\n\n", "THE END"],  # Multiple stop sequences
    presence_penalty=0.3,  # Encourage novel elements
    frequency_penalty=0.3,  # Reduce repetitive phrases
    n=1,  # Generate one completion
    seed=123,  # For reproducible results
)

print_highlight(f"Response: {response}")

[2024-11-04 09:44:20 TP0] Prefill batch. #new-seq: 1, #new-token: 9, #cached-token: 1, cache hit rate: 31.35%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2024-11-04 09:44:20 TP0] Decode batch. #running-req: 1, #token: 13, token usage: 0.00, gen throughput (token/s): 130.98, #queue-req: 0


[2024-11-04 09:44:21 TP0] Decode batch. #running-req: 1, #token: 53, token usage: 0.00, gen throughput (token/s): 135.58, #queue-req: 0


[2024-11-04 09:44:21 TP0] Decode batch. #running-req: 1, #token: 93, token usage: 0.00, gen throughput (token/s): 134.60, #queue-req: 0


[2024-11-04 09:44:21 TP0] Decode batch. #running-req: 1, #token: 133, token usage: 0.00, gen throughput (token/s): 133.51, #queue-req: 0
[2024-11-04 09:44:21] INFO:     127.0.0.1:48578 - "POST /v1/completions HTTP/1.1" 200 OK


## Structured decoding (JSON, Regex)
You can specify a JSON schema or a regular expression to constrain the model output. The model output will be guaranteed to follow the given constraints.

### JSON

In [7]:
import json

json_schema = json.dumps(
    {
        "type": "object",
        "properties": {
            "name": {"type": "string", "pattern": "^[\\w]+$"},
            "population": {"type": "integer"},
        },
        "required": ["name", "population"],
    }
)

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "user", "content": "Give me the information of the capital of France in the JSON format."},
    ],
    temperature=0,
    max_tokens=128,
    response_format={
        "type": "json_schema",
        "json_schema": {"name": "foo", "schema": json.loads(json_schema)},
    },
)

print_highlight(response.choices[0].message.content)

[2024-11-04 09:44:22 TP0] Prefill batch. #new-seq: 1, #new-token: 19, #cached-token: 30, cache hit rate: 37.61%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2024-11-04 09:44:22 TP0] Decode batch. #running-req: 1, #token: 0, token usage: 0.00, gen throughput (token/s): 75.22, #queue-req: 0
[2024-11-04 09:44:22] INFO:     127.0.0.1:48578 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### Regular expression

In [8]:
response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "user", "content": "What is the capital of France?"},
    ],
    temperature=0,
    max_tokens=128,
    extra_body={"regex": "(Paris|London)"},
)

print_highlight(response.choices[0].message.content)

[2024-11-04 09:44:22 TP0] Prefill batch. #new-seq: 1, #new-token: 12, #cached-token: 30, cache hit rate: 42.75%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2024-11-04 09:44:22] INFO:     127.0.0.1:48578 - "POST /v1/chat/completions HTTP/1.1" 200 OK


## Batches

Batches API for chat completions and completions are also supported. You can upload your requests in `jsonl` files, create a batch job, and retrieve the results when the batch job is completed (which takes longer but costs less).

The batches APIs are:

- `batches`
- `batches/{batch_id}/cancel`
- `batches/{batch_id}`

Here is an example of a batch job for chat completions, completions are similar.


In [9]:
import json
import time
from openai import OpenAI

client = OpenAI(base_url="http://127.0.0.1:30000/v1", api_key="None")

requests = [
    {
        "custom_id": "request-1",
        "method": "POST",
        "url": "/chat/completions",
        "body": {
            "model": "meta-llama/Meta-Llama-3.1-8B-Instruct",
            "messages": [
                {"role": "user", "content": "Tell me a joke about programming"}
            ],
            "max_tokens": 50,
        },
    },
    {
        "custom_id": "request-2",
        "method": "POST",
        "url": "/chat/completions",
        "body": {
            "model": "meta-llama/Meta-Llama-3.1-8B-Instruct",
            "messages": [{"role": "user", "content": "What is Python?"}],
            "max_tokens": 50,
        },
    },
]

input_file_path = "batch_requests.jsonl"

with open(input_file_path, "w") as f:
    for req in requests:
        f.write(json.dumps(req) + "\n")

with open(input_file_path, "rb") as f:
    file_response = client.files.create(file=f, purpose="batch")

batch_response = client.batches.create(
    input_file_id=file_response.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
)

print_highlight(f"Batch job created with ID: {batch_response.id}")

[2024-11-04 09:44:22] INFO:     127.0.0.1:48622 - "POST /v1/files HTTP/1.1" 200 OK
[2024-11-04 09:44:22] INFO:     127.0.0.1:48622 - "POST /v1/batches HTTP/1.1" 200 OK
[2024-11-04 09:44:22 TP0] Prefill batch. #new-seq: 2, #new-token: 18, #cached-token: 62, cache hit rate: 50.56%, token usage: 0.00, #running-req: 0, #queue-req: 0


In [10]:
while batch_response.status not in ["completed", "failed", "cancelled"]:
    time.sleep(3)
    print(f"Batch job status: {batch_response.status}...trying again in 3 seconds...")
    batch_response = client.batches.retrieve(batch_response.id)

if batch_response.status == "completed":
    print("Batch job completed successfully!")
    print(f"Request counts: {batch_response.request_counts}")

    result_file_id = batch_response.output_file_id
    file_response = client.files.content(result_file_id)
    result_content = file_response.read().decode("utf-8")

    results = [
        json.loads(line) for line in result_content.split("\n") if line.strip() != ""
    ]

    for result in results:
        print_highlight(f"Request {result['custom_id']}:")
        print_highlight(f"Response: {result['response']}")

    print_highlight("Cleaning up files...")
    # Only delete the result file ID since file_response is just content
    client.files.delete(result_file_id)
else:
    print_highlight(f"Batch job failed with status: {batch_response.status}")
    if hasattr(batch_response, "errors"):
        print_highlight(f"Errors: {batch_response.errors}")

[2024-11-04 09:44:22 TP0] Decode batch. #running-req: 1, #token: 78, token usage: 0.00, gen throughput (token/s): 93.10, #queue-req: 0


Batch job status: validating...trying again in 3 seconds...
[2024-11-04 09:44:25] INFO:     127.0.0.1:48622 - "GET /v1/batches/batch_32dd340d-dedc-4503-8036-720f7d60a78d HTTP/1.1" 200 OK
Batch job completed successfully!
Request counts: BatchRequestCounts(completed=2, failed=0, total=2)
[2024-11-04 09:44:25] INFO:     127.0.0.1:48622 - "GET /v1/files/backend_result_file-d43fba1a-ff77-4c80-b6e5-88f2d436e13a/content HTTP/1.1" 200 OK


[2024-11-04 09:44:25] INFO:     127.0.0.1:48622 - "DELETE /v1/files/backend_result_file-d43fba1a-ff77-4c80-b6e5-88f2d436e13a HTTP/1.1" 200 OK


It takes a while to complete the batch job. You can use these two APIs to retrieve the batch job status or cancel the batch job.

1. `batches/{batch_id}`: Retrieve the batch job status.
2. `batches/{batch_id}/cancel`: Cancel the batch job.

Here is an example to check the batch job status.

In [11]:
import json
import time
from openai import OpenAI

client = OpenAI(base_url="http://127.0.0.1:30000/v1", api_key="None")

requests = []
for i in range(100):
    requests.append(
        {
            "custom_id": f"request-{i}",
            "method": "POST",
            "url": "/chat/completions",
            "body": {
                "model": "meta-llama/Meta-Llama-3.1-8B-Instruct",
                "messages": [
                    {
                        "role": "system",
                        "content": f"{i}: You are a helpful AI assistant",
                    },
                    {
                        "role": "user",
                        "content": "Write a detailed story about topic. Make it very long.",
                    },
                ],
                "max_tokens": 500,
            },
        }
    )

input_file_path = "batch_requests.jsonl"
with open(input_file_path, "w") as f:
    for req in requests:
        f.write(json.dumps(req) + "\n")

with open(input_file_path, "rb") as f:
    uploaded_file = client.files.create(file=f, purpose="batch")

batch_job = client.batches.create(
    input_file_id=uploaded_file.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
)

print_highlight(f"Created batch job with ID: {batch_job.id}")
print_highlight(f"Initial status: {batch_job.status}")

time.sleep(10)

max_checks = 5
for i in range(max_checks):
    batch_details = client.batches.retrieve(batch_id=batch_job.id)

    print_highlight(
        f"Batch job details (check {i+1} / {max_checks}) // ID: {batch_details.id} // Status: {batch_details.status} // Created at: {batch_details.created_at} // Input file ID: {batch_details.input_file_id} // Output file ID: {batch_details.output_file_id}"
    )
    print_highlight(
        f"<strong>Request counts: Total: {batch_details.request_counts.total} // Completed: {batch_details.request_counts.completed} // Failed: {batch_details.request_counts.failed}</strong>"
    )

    time.sleep(3)

[2024-11-04 09:44:25] INFO:     127.0.0.1:48658 - "POST /v1/files HTTP/1.1" 200 OK
[2024-11-04 09:44:25] INFO:     127.0.0.1:48658 - "POST /v1/batches HTTP/1.1" 200 OK


[2024-11-04 09:44:25 TP0] Prefill batch. #new-seq: 17, #new-token: 510, #cached-token: 425, cache hit rate: 46.86%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2024-11-04 09:44:25 TP0] Prefill batch. #new-seq: 83, #new-token: 2490, #cached-token: 2075, cache hit rate: 45.77%, token usage: 0.00, #running-req: 17, #queue-req: 0


[2024-11-04 09:44:25 TP0] Decode batch. #running-req: 100, #token: 6025, token usage: 0.01, gen throughput (token/s): 934.12, #queue-req: 0


[2024-11-04 09:44:26 TP0] Decode batch. #running-req: 100, #token: 10025, token usage: 0.02, gen throughput (token/s): 10813.24, #queue-req: 0


[2024-11-04 09:44:26 TP0] Decode batch. #running-req: 100, #token: 14025, token usage: 0.03, gen throughput (token/s): 10600.46, #queue-req: 0


...

[2024-11-04 09:44:35] INFO:     127.0.0.1:43906 - "GET /v1/batches/batch_c82e1a7b-5c50-4ec1-82a1-37b6da214b63 HTTP/1.1" 200 OK


[2024-11-04 09:44:38] INFO:     127.0.0.1:43906 - "GET /v1/batches/batch_c82e1a7b-5c50-4ec1-82a1-37b6da214b63 HTTP/1.1" 200 OK


[2024-11-04 09:44:41] INFO:     127.0.0.1:43906 - "GET /v1/batches/batch_c82e1a7b-5c50-4ec1-82a1-37b6da214b63 HTTP/1.1" 200 OK


[2024-11-04 09:44:44] INFO:     127.0.0.1:43906 - "GET /v1/batches/batch_c82e1a7b-5c50-4ec1-82a1-37b6da214b63 HTTP/1.1" 200 OK


[2024-11-04 09:44:47] INFO:     127.0.0.1:43906 - "GET /v1/batches/batch_c82e1a7b-5c50-4ec1-82a1-37b6da214b63 HTTP/1.1" 200 OK


Here is an example to cancel a batch job.

In [12]:
import json
import time
from openai import OpenAI
import os

client = OpenAI(base_url="http://127.0.0.1:30000/v1", api_key="None")

requests = []
for i in range(500):
    requests.append(
        {
            "custom_id": f"request-{i}",
            "method": "POST",
            "url": "/chat/completions",
            "body": {
                "model": "meta-llama/Meta-Llama-3.1-8B-Instruct",
                "messages": [
                    {
                        "role": "system",
                        "content": f"{i}: You are a helpful AI assistant",
                    },
                    {
                        "role": "user",
                        "content": "Write a detailed story about topic. Make it very long.",
                    },
                ],
                "max_tokens": 500,
            },
        }
    )

input_file_path = "batch_requests.jsonl"
with open(input_file_path, "w") as f:
    for req in requests:
        f.write(json.dumps(req) + "\n")

with open(input_file_path, "rb") as f:
    uploaded_file = client.files.create(file=f, purpose="batch")

batch_job = client.batches.create(
    input_file_id=uploaded_file.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
)

print_highlight(f"Created batch job with ID: {batch_job.id}")
print_highlight(f"Initial status: {batch_job.status}")

time.sleep(10)

try:
    cancelled_job = client.batches.cancel(batch_id=batch_job.id)
    print_highlight(f"Cancellation initiated. Status: {cancelled_job.status}")
    assert cancelled_job.status == "cancelling"

    # Monitor the cancellation process
    while cancelled_job.status not in ["failed", "cancelled"]:
        time.sleep(3)
        cancelled_job = client.batches.retrieve(batch_job.id)
        print_highlight(f"Current status: {cancelled_job.status}")

    # Verify final status
    assert cancelled_job.status == "cancelled"
    print_highlight("Batch job successfully cancelled")

except Exception as e:
    print_highlight(f"Error during cancellation: {e}")
    raise e

finally:
    try:
        del_response = client.files.delete(uploaded_file.id)
        if del_response.deleted:
            print_highlight("Successfully cleaned up input file")
        if os.path.exists(input_file_path):
            os.remove(input_file_path)
            print_highlight("Successfully deleted local batch_requests.jsonl file")
    except Exception as e:
        print_highlight(f"Error cleaning up: {e}")
        raise e

[2024-11-04 09:44:50] INFO:     127.0.0.1:47616 - "POST /v1/files HTTP/1.1" 200 OK
[2024-11-04 09:44:50] INFO:     127.0.0.1:47616 - "POST /v1/batches HTTP/1.1" 200 OK


[2024-11-04 09:44:50 TP0] Prefill batch. #new-seq: 37, #new-token: 37, #cached-token: 1998, cache hit rate: 59.28%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2024-11-04 09:44:50 TP0] Prefill batch. #new-seq: 334, #new-token: 8192, #cached-token: 10177, cache hit rate: 56.57%, token usage: 0.01, #running-req: 37, #queue-req: 129


[2024-11-04 09:44:51 TP0] Prefill batch. #new-seq: 130, #new-token: 3871, #cached-token: 3279, cache hit rate: 54.28%, token usage: 0.03, #running-req: 370, #queue-req: 1


[2024-11-04 09:44:51 TP0] Decode batch. #running-req: 500, #token: 20525, token usage: 0.05, gen throughput (token/s): 398.92, #queue-req: 0


[2024-11-04 09:44:52 TP0] Decode batch. #running-req: 500, #token: 40525, token usage: 0.09, gen throughput (token/s): 24640.81, #queue-req: 0


...

[2024-11-04 09:44:59 TP0] Decode batch. #running-req: 500, #token: 180525, token usage: 0.41, gen throughput (token/s): 18496.75, #queue-req: 0


[2024-11-04 09:45:00 TP0] Decode batch. #running-req: 500, #token: 200525, token usage: 0.45, gen throughput (token/s): 17846.55, #queue-req: 0


[2024-11-04 09:45:00] INFO:     127.0.0.1:33408 - "POST /v1/batches/batch_9cc4f64b-c9ee-4b41-b15b-1db1f5d3f69f/cancel HTTP/1.1" 200 OK


[2024-11-04 09:45:03] INFO:     127.0.0.1:33408 - "GET /v1/batches/batch_9cc4f64b-c9ee-4b41-b15b-1db1f5d3f69f HTTP/1.1" 200 OK


[2024-11-04 09:45:03] INFO:     127.0.0.1:33408 - "DELETE /v1/files/backend_input_file-403344e2-be58-4ea1-a321-80265e3ade09 HTTP/1.1" 200 OK


In [13]:
terminate_process(server_process)